# 8. 일급함수, 익명함수, callable, 매개변수, signature, partial

### 일급함수 (일급 객체) - 파이썬 함수의 특징 
1. 런타임 초기화 가능 
2. 변수에 할당 가능 
3. 함수 인수에 전달 가능 (ex) sorted(keys=len) - 함수 안에 함수 
4. 함수 결과로 반환 가능 (ex) return function 

## (1) 팩토리얼 함수 구현 예제